In [1]:
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
from sklearn.cluster import KMeans
import cartopy.crs as ccrs
import matplotlib.patches as mpatches
from shapely.affinity import translate

In [2]:
# Reading the file with metrices
df_parking = gpd.read_file("F:/Berlin-DOP_2020/Processed images_new/Predictions.GPKG", crs='EPSG:25833', Layer= "Predict_with_info", driver="GPKG")

In [3]:
# Renaming the column heading
df_parking = df_parking.rename(columns={'True or False': 'True_or_False'})

In [4]:
# Adding a buffer of 1 meters to parking polygons
df_parking.geometry = df_parking.geometry.buffer(1)

In [5]:
df_parking

,index_right,id,label,parking_area_m2,geometry
0,8602,8608,PK-space-occupied,48.033860,"POLYGON ((392967.956 5821003.838, 392967.956 5..."
1,8600,8606,PK-space-empty,48.033860,"POLYGON ((392967.956 5821003.838, 392967.956 5..."
2,8601,8607,PK-space-occupied,48.033860,"POLYGON ((392967.956 5821003.838, 392967.956 5..."
3,8597,8603,PK-space-occupied,48.033860,"POLYGON ((392967.956 5821003.838, 392967.956 5..."
4,1963,1968,PK-space-occupied,14.047920,"POLYGON ((392135.950 5820998.151, 392135.900 5..."
...,...,...,...,...,...
8971,6026,6032,PK-space-occupied,10.368876,"POLYGON ((392624.119 5824996.170, 392624.071 5..."
8972,5129,5135,PK-space-occupied,34.782663,"POLYGON ((392538.454 5824993.451, 392538.400 5..."
8973,3069,3075,PK-space-occupied,33.143099,"POLYGON ((392242.851 5824996.771, 392242.736 5..."
8974,517,521,PK-space-occupied,23.646757,"POLYGON ((391527.381 5825001.729, 391529.477 5..."


In [6]:
# Creating a new fataframe that has union of overlapping geometries
geoms = df_parking.geometry.unary_union
df = gpd.GeoDataFrame(geometry=[geoms])
df = df.explode().reset_index(drop=True)
df.set_geometry = df.geometry
df.crs = {'init' :'epsg:25833'}

C:\ProgramData\Anaconda3\envs\geopandas\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [7]:
df.crs

<Projected CRS: EPSG:25833>
Name: ETRS89 / UTM zone 33N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Europe - 12°E to 18°E and ETRS89 by country
- bounds: (12.0, 46.4, 18.01, 84.01)
Coordinate Operation:
- name: UTM zone 33N
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [9]:
# Taking the spatial join of the dataframes
merged_df = gpd.sjoin(df,df_parking, how="inner")

merged_df["parking_area_m2"] = merged_df['geometry'].area
# Saving the file
merged_df.to_file("F:/Berlin-DOP_2020/Processed images_new/parking_dist.gpkg", driver="GPKG", layer="parking_disolved_withdata")

ValueError: 'index_left' and 'index_right' cannot be names in the frames being joined

In [ ]:
# Saving the file
merged_df.to_file("F:/Berlin-DOP_2020/Processed images_new/Predictions.GPKG", crs='EPSG:25833', Layer= "Predict_dissolved", driver="GPKG")